<a href="https://colab.research.google.com/github/bhargavi-nakathe/bhargavi-nakathe/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install  --quiet --upgrade langchain-text-splitters langchain-community
#%pip install langchain_mistralai
%pip install -qU langchain-chroma
%pip install langchain-groq
%pip install langchain-huggingface
%pip install pymupdf pymupdf4llm
%pip install "unstructured[md]" nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:0

In [ ]:
#Load the Drive helper and mount
from google.colab import drive

#this will prompt for authorisation
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/dataset/"

HarryPotter1.pdf


In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
import pymupdf4llm
import pathlib


docs = pymupdf4llm.to_markdown('/content/drive/My Drive/dataset/HarryPotter1.pdf')
pathlib.Path("/content/drive/My Drive/dataset/HarryPotter1.md").write_bytes(docs.encode())
markdown_path  = '/content/drive/My Drive/dataset/HarryPotter1.md'
loader  = UnstructuredMarkdownLoader(markdown_path)


docs = loader.load()


readme_content = docs[0].page_content


print(readme_content[:250])
assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")



Processing /content/drive/My Drive/dataset/HarryPotter1.pdf...
[                                        ] (0/250)[                                        ] (  1/250)[                                        ] (  2/250)[                                        ] (  3/250)[                                        ] (  4/250)[                                        ] (  5/250)[                

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1000,  # chunk size (characters)
   chunk_overlap=200,  # chunk overlap (characters)
   add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)


print(f"Split Document into {len(all_splits)} sub-documents.")



Split Document into 572 sub-documents.


In [ ]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0,
    streaming=True
)


In [ ]:
import getpass
import os


from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}


embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)




In [ ]:
from langchain_chroma import Chroma
import time

vector_store = Chroma(embedding_function=embeddings)

for i in range(0, len(all_splits), 10):  # Process chunks in batches of 10
    batch = all_splits[i : i + 10]
    document_ids = vector_store.add_documents(documents=batch)
    print(f"Added documents {i}-{i + len(batch)}")
    # time.sleep(5)

# document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

Added documents 0-10
Added documents 10-20
Added documents 20-30
Added documents 30-40
Added documents 40-50
Added documents 50-60
Added documents 60-70
Added documents 70-80
Added documents 80-90
Added documents 90-100
Added documents 100-110
Added documents 110-120
Added documents 120-130
Added documents 130-140
Added documents 140-150
Added documents 150-160
Added documents 160-170
Added documents 170-180
Added documents 180-190
Added documents 190-200
Added documents 200-210
Added documents 210-220
Added documents 220-230
Added documents 230-240
Added documents 240-250
Added documents 250-260
Added documents 260-270
Added documents 270-280
Added documents 280-290
Added documents 290-300
Added documents 300-310
Added documents 310-320
Added documents 320-330
Added documents 330-340
Added documents 340-350
Added documents 350-360
Added documents 360-370
Added documents 370-380
Added documents 380-390
Added documents 390-400
Added documents 400-410
Added documents 410-420
Added docume

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences and keep the answer concise.
{context}
Question: {question}
Helpful Answer:
"""

question = input("Ask a question: ")

retrieved_docs = vector_store.similarity_search(question, k=3)
# print(f"Found {len(retrieved_docs)} documents")

context = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | llm

response = chain.invoke({"context": context, "question": question}).content
print("Response: ", response)

Ask a question: did hermoine like ron weaseley
Response:  Hermione Granger and Ron Weasley have a complex relationship, but in this context, it doesn't explicitly show that she likes him romantically. Hermione is portrayed as being bossy and interfering, which frustrates Ron, but it doesn't necessarily indicate her feelings towards him. It seems that Hermione is more focused on achieving her goals and helping herridged friends than on having romantic feelings for Ron at this point.
